# Bitcoin vs. Ethereum
This project is conducted by Firebolt, Inc. All Rights Reserved. Licensed under the MIT License.
```
The MIT License

Copyright (c) 2019 - Present Firebolt, Inc.

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.
```

## Step 1. Import essential packages, create essential variables, and load the dataset.

## Step 1.1. Import essential packages.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import time
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout

Using TensorFlow backend.


## Step 1.2. Load the dataset and create essential variables.
We will be using the ```https://coinmarketcap.com/currencies/bitcoin/historical-data/``` dataset for Bitcoin and ```https://coinmarketcap.com/currencies/ethereum/historical-data/``` for Ethereum.

In [2]:
# Load the coinmarketcap.com Bitcoin data
bitcoin_market_info = pd.read_html("https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20130428&end="+time.strftime("%Y%m%d"))[0]
# Convert the date string to the correct format
bitcoin_market_info = bitcoin_market_info.assign(Date=pd.to_datetime(bitcoin_market_info['Date']))
# If volume is '-', then change it into a readable format by changing it to 0
bitcoin_market_info.loc[bitcoin_market_info['Volume']=="-",'Volume']=0
# Convert the bitcoin_market_info variable to int
bitcoin_market_info['Volume'] = bitcoin_market_info['Volume'].astype('int64')
# Look at the first 5 rows to make sure that the data is loaded successfully
bitcoin_market_info.head()

,Date,Open*,High,Low,Close**,Volume,Market Cap
0,2019-05-13,6971.18,8047.41,6898.28,7814.92,28677672181,138316573807
1,2019-05-12,7203.51,7503.87,6815.77,6972.37,27773333680,123389710222
2,2019-05-11,6379.67,7333.00,6375.70,7204.77,28867562329,127488435181
3,2019-05-10,6175.82,6434.62,6161.52,6378.85,19419875368,112863284035
4,2019-05-09,5982.32,6183.04,5982.32,6174.53,16784645411,109238600750


### Now ethereum!

In [4]:
# Load marketcap.com Ethereum data
ethereum_market_info = pd.read_html("https://coinmarketcap.com/currencies/ethereum/historical-data/?start=20130428&end="+time.strftime("%Y%m%d"))[0]
# Convert the date string to the correct format
ethereum_market_info = ethereum_market_info.assign(Date=pd.to_datetime(ethereum_market_info['Date']))
# If volume is '-', then change it into a readable format by changing it to 0
ethereum_market_info.columns = ethereum_market_info.columns.str.replace("*", "")
# Look at the first 5 rows to make sure that the data is loaded successfully
ethereum_market_info.head()

,Date,Open,High,Low,Close,Volume,Market Cap
0,2019-05-13,187.42,205.10,185.31,196.85,10367149039,20876934045
1,2019-05-12,194.16,201.88,183.65,187.33,10326050090,19864635091
2,2019-05-11,173.14,203.47,173.14,194.30,12578331654,20602050120
3,2019-05-10,170.31,175.89,168.99,173.14,8036919053,18355936406
4,2019-05-09,170.95,172.88,167.25,170.29,6546914198,18051087500


# Step 2. Visualization & Plotting

In [5]:
### LOAD BITCOIN & ETHEREUM LOGOS ###
import sys
from PIL import Image
import io

if sys.version_info[0] < 3:
    import urllib2 as urllib
    bt_img = urllib.urlopen("http://logok.org/wp-content/uploads/2016/10/Bitcoin-Logo-640x480.png")
    eth_img = urllib.urlopen("https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Ethereum_logo_2014.svg/256px-Ethereum_logo_2014.svg.png")
else:
    import urllib
    bt_img = urllib.request.urlopen("http://logok.org/wp-content/uploads/2016/10/Bitcoin-Logo-640x480.png")
    eth_img = urllib.request.urlopen("https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Ethereum_logo_2014.svg/256px-Ethereum_logo_2014.svg.png")

image_file = io.BytesIO(bt_img.read())
bitcoin_im = Image.open(image_file)

image_file = io.BytesIO(eth_img.read())
eth_im = Image.open(image_file)
width_eth_im , height_eth_im  = eth_im.size
eth_im = eth_im.resize((int(eth_im.size[0]*0.8), int(eth_im.size[1]*0.8)), Image.ANTIALIAS)

In [6]:
bitcoin_market_info.columns =[bitcoin_market_info.columns[0]]+['bt_'+i for i in bitcoin_market_info.columns[1:]]
ethereum_market_info.columns =[ethereum_market_info.columns[0]]+['eth_'+i for i in ethereum_market_info.columns[1:]]

In [9]:
market_info = pd.merge(bitcoin_market_info,ethereum_market_info, on=['Date'])
market_info = market_info[market_info['Date']>='2016-01-01']
for coins in ['bt_', 'eth_']: 
    kwargs = { coins+'day_diff': lambda x: (x[coins+'Close']-x[coins+'Open'])/x[coins+'Open']}
    market_info = market_info.assign(**kwargs)
market_info.head()

KeyError: 'bt_Close'